In [1]:
import pandas as pd
import pyspark
import matplotlib


In [2]:
import os
from pyspark.sql import SQLContext, SparkSession

In [5]:
spark = SparkSession\
    .builder\
        .master('spark://192.168.68.106:7077')\
            .appName("sparkFromJupiter")\
                .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sparkContext=sc, sparkSession=spark)

23/04/30 13:32:09 WARN Utils: Your hostname, Sorawits-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.100 instead (on interface en0)
23/04/30 13:32:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/30 13:32:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/30 13:32:29 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
23/04/30 13:33:10 WARN StandaloneSchedulerBackend: Application ID is not initialized yet.
23/04/30 13:33:10 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: All masters are unresponsive!

23/04/30 13:33:10 WARN StandaloneAppClient$ClientEndpoint: Drop UnregisterApplication(null) because has not yet connected to master


In [6]:
sc

<SparkContext master=spark://192.168.68.106:7077 appName=sparkFromJupiter>